| **Author**          | **Roll No**   | **Version** |
|---------------------|----------------|--------------|
| Kaloori Shiva Prasad       | 24250041      | 1.0          |


In [ ]:
import requests
from bs4 import BeautifulSoup
import zipfile
import os
import re

base_url = "https://www.msubs.net"


def download_txt(download_url):
    response = requests.get(download_url)
    response.raise_for_status()  

    
    soup = BeautifulSoup(response.text, 'html.parser')

   
    script_tag = soup.find('script', string=re.compile(r's2'))

    if script_tag:
        script_content = script_tag.string
        print("JavaScript content found in the HTML.")

        
        s1_match = re.search(r"var s1\s*=\s*'(.+?)';", script_content)
        s2_match = re.search(r"var s2\s*=\s*'(.+?)';", script_content)
        s3_match = re.search(r"var s3\s*=\s*'(.+?)';", script_content)
        s4_match = re.search(r"var s4\s*=\s*'(.+?)';", script_content)

        if s1_match and s2_match and s3_match and s4_match:
            s1 = s1_match.group(1)
            s2 = s2_match.group(1)
            s3 = s3_match.group(1)
            s4 = s4_match.group(1)

            
            constructed_url = f"{s1}{s2}{s3}{s4}".replace(" ", "%20")

            
            if not constructed_url.startswith("http"):
                constructed_url = f"https://www.msubs.net/{constructed_url}"

            print(f"Constructed ZIP file download URL: {constructed_url}")

            
            zip_file_response = requests.get(constructed_url)
            zip_file_response.raise_for_status()

            
            zip_filename = s4 
            with open(zip_filename, 'wb') as f:
                f.write(zip_file_response.content)
            print(f"ZIP file '{zip_filename}' downloaded successfully.")

            
            extract_folder = '/mnt/HDFS1/language_nlp/english_nlp_team17/Assignment1/Data_Shiva/Subtitles'
            os.makedirs(extract_folder, exist_ok=True)

            
            try:
                with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
                    zip_ref.testzip()  
                    zip_ref.extractall(extract_folder)
                print("ZIP file extracted successfully.")

                
                for file in os.listdir(extract_folder):
                    if file.endswith('.srt'):
                        srt_file_path = os.path.join(extract_folder, file)
                        txt_file_path = srt_file_path.replace('.srt', '.txt')
                        
                        srt_to_txt(srt_file_path, txt_file_path)
                        
                        
                        os.remove(srt_file_path)
                        print(f"Deleted {srt_file_path} after conversion.")
                
                
                os.remove(zip_filename)
                print(f"ZIP file '{zip_filename}' deleted.")

            except zipfile.BadZipFile:
                print("The downloaded file is not a valid ZIP file.")
        else:
            print("Failed to extract 's1', 's2', 's3', or 's4' from the script.")
    else:
        print("No JavaScript content with 's2' found on the page.")


def srt_to_txt(srt_file_path, txt_file_path):
    
    encodings = ['utf-8', 'iso-8859-1', 'cp1252']
    for encoding in encodings:
        try:
            with open(srt_file_path, 'r', encoding=encoding) as srt_file:
                with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
                    for line in srt_file:
                        
                        if '-->' in line or re.match(r'^\d+$', line.strip()):
                            continue
                        
                        txt_file.write(line)
            print(f"Subtitle converted to text and saved as {txt_file_path}")
            break
        except UnicodeDecodeError:
            print(f"Failed to decode {srt_file_path} with encoding {encoding}. Trying next encoding.")
    

for i in range(1, 421):  
    download_url = f"https://www.msubs.net//download-{i}.html"
    download_txt(download_url)
